In [1]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
import pickle
import tqdm
from time import time

In [2]:
import multiprocessing
multiprocessing.cpu_count()

80

In [3]:
# Y_Xs_count_dict=pickle.load(open("0707_sell_data_dict.pickle","rb"))
Y_Xs_type_count_dict_365567=pickle.load(open("aaa.pickle","rb"))
Xs_typ_table=pickle.load(open("aaa.pickle","rb"))

In [ ]:
len(Y_Xs_count_dict)

In [ ]:
len(Y_Xs_type_count_dict)

In [ ]:
for i in Y_Xs_type_count_dict :
    try:
        pd.to_numeric(i)
    except:
        print(i)
        break

In [ ]:
for i in Y_Xs_count_dict :
    print(i,Y_Xs_count_dict[i])
    break

In [ ]:
for i in Y_Xs_type_count_dict :
    print(i,Y_Xs_type_count_dict[i])
    break

In [ ]:
Xs_typ_table.head()

In [4]:
num_cores = 50

In [5]:
def df_making(dict_):
    df = pd.DataFrame(columns=list(Xs_typ_table['big_typ'].unique()))
    index = []
    for i in tqdm.tqdm(dict_) :
        data = Y_Xs_type_count_dict_365567[i]
        df = df.append(data,ignore_index=True)
        index.append(i)
    df.index=index
    return df

def split_dict(input_dict: dict, num_parts: int) -> list:
    list_len: int = len(input_dict)
    return [dict(list(input_dict.items())[i * list_len // num_parts:(i + 1) * list_len // num_parts])
        for i in range(num_parts)]

def parallelize(dic, func):
    dic_split = split_dict(dic, num_cores)
    pool = Pool(num_cores)
    result = pd.concat(pool.map(func, dic_split))
    pool.close()
    pool.join()
    return result

In [6]:
data=split_dict(Y_Xs_type_count_dict_365567,2)

In [7]:
data1=split_dict(data[0],2)
data2=split_dict(data[1],2)

In [8]:
data1_1=data1[0]
data1_2=data1[1]
data2_1=data2[0]
data2_2=data2[1]

In [9]:
start=time() # 353.9193482398987 # 8.846171140670776
result1=parallelize(data1_1,df_making)
print(time()-start)

start=time() #285.6164927482605 # 8.900442600250244
result2=parallelize(data1_2,df_making)
print(time()-start)

100%|██████████| 1828/1828 [00:05<00:00, 318.68it/s]

100%|██████████| 1828/1828 [00:06<00:00, 284.28it/s]


11.123480081558228


100%|██████████| 1828/1828 [00:04<00:00, 368.11it/s]

100%|██████████| 1827/1827 [00:05<00:00, 323.76it/s]


10.327222347259521


In [10]:
start=time() # 254.7865993976593 # 8.944982528686523
result3=parallelize(data2_1,df_making)
print(time()-start)

start=time() # 360.2212383747101 # 9.098328351974487
result4=parallelize(data2_2,df_making)
print(time()-start)

100%|██████████| 1828/1828 [00:05<00:00, 341.49it/s]

 76%|███████▌  | 1386/1827 [00:05<00:01, 235.09it/s]

100%|██████████| 1828/1828 [00:05<00:00, 329.69it/s]

100%|██████████| 1828/1828 [00:06<00:00, 296.95it/s]


10.584693431854248


 93%|█████████▎| 1704/1828 [00:05<00:00, 410.82it/s]

100%|██████████| 1828/1828 [00:05<00:00, 322.97it/s]

 84%|████████▍ | 1542/1828 [00:05<00:00, 400.00it/s]

100%|██████████| 1827/1827 [00:05<00:00, 313.67it/s]

100%|██████████| 1828/1828 [00:06<00:00, 299.37it/s]


10.481066226959229


In [11]:
print(result1.shape)
print(result2.shape)
print(result3.shape)
print(result4.shape)

(91391, 20)
(91392, 20)
(91392, 20)
(91392, 20)


In [12]:
def df_1_making(df):
    df_1=pd.DataFrame(columns=df.columns)
    for i in tqdm.tqdm(range(len(df))):
        df_1=df_1.append(df.iloc[i]/df.iloc[i].sum())
    df_1.index=df.index
    return df_1

def parallelize(df, func):
    df_split=np.array_split(df, num_cores) 
    pool = Pool(num_cores)
    result = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return result

In [13]:
start=time() # 121.86266326904297 # 7.789303779602051
df_1_1=parallelize(result1,df_1_making)
time()-start

100%|██████████| 1828/1828 [00:06<00:00, 281.05it/s]

100%|██████████| 1827/1827 [00:06<00:00, 280.80it/s]

100%|██████████| 1828/1828 [00:06<00:00, 279.21it/s]

100%|██████████| 1827/1827 [00:07<00:00, 257.60it/s]


7.789303779602051

In [14]:
start=time() # 90.73159718513489 # 7.353627681732178
df_1_2=parallelize(result2,df_1_making)
time()-start

100%|██████████| 1828/1828 [00:05<00:00, 323.74it/s]

100%|██████████| 1828/1828 [00:05<00:00, 317.83it/s]

100%|██████████| 1828/1828 [00:06<00:00, 296.37it/s]

100%|██████████| 1827/1827 [00:06<00:00, 295.56it/s]

100%|██████████| 1828/1828 [00:06<00:00, 272.99it/s]


7.353627681732178

In [15]:
start=time() # 91.24309062957764 # 7.109282970428467
df_1_3=parallelize(result3,df_1_making)
time()-start

100%|██████████| 1828/1828 [00:05<00:00, 323.78it/s]

100%|██████████| 1828/1828 [00:05<00:00, 305.32it/s]

100%|██████████| 1828/1828 [00:06<00:00, 281.76it/s]


7.109282970428467

In [16]:
start=time() #87.9748957157135 # 8.001725196838379
df_1_4=parallelize(result4,df_1_making)
time()-start

100%|██████████| 1828/1828 [00:06<00:00, 293.12it/s]

100%|██████████| 1828/1828 [00:06<00:00, 278.46it/s]

100%|██████████| 1828/1828 [00:06<00:00, 268.30it/s]

100%|██████████| 1827/1827 [00:07<00:00, 254.46it/s]


8.001725196838379

In [ ]:
df_1=pd.concat([df_1_1,df_1_2,df_1_3,df_1_4])

In [18]:
df_1.fillna(0,inplace=True)

In [19]:
df_1

,숙박 및 음식점업,도매 및 소매업,건설업,전문 과학 및 기술서비스업,부동산업 및 임대업,농업 임업 및 어업,전기 가스 증기 및 수도사업,제조업,보건업 및 사회복지서비스업,하수폐기물처리 원료재생 및 환경복원,NaN,교육서비스업,출판 영상 방송통신 및 정보서비스업,예술 스포츠 및 여가관련 서비스업,금융 및 보험업,운수업,사업시설관리 및 사업지원서비스업,협회 및 단체 수리 및 기타 개인서비스,가구내고용활동,광업
6058191362,0.0,0.031496,0.000000,0.307087,0.000000,0.0,0.000000,0.094488,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.566929,0.000000,0.000000,0.0,0.0
6178603207,0.0,0.001812,0.000000,0.000000,0.016304,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.961957,0.001812,0.018116,0.0,0.0
4098610941,0.0,0.406780,0.144068,0.046610,0.004237,0.0,0.033898,0.152542,0.0,0.0,0.088983,0.0,0.033898,0.0,0.0,0.088983,0.000000,0.000000,0.0,0.0
6038171958,0.0,0.620690,0.000000,0.241379,0.000000,0.0,0.000000,0.068966,0.0,0.0,0.034483,0.0,0.000000,0.0,0.0,0.034483,0.000000,0.000000,0.0,0.0
1288713240,0.0,0.496503,0.000000,0.000000,0.000000,0.0,0.000000,0.489510,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.013986,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8614100470,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
9227588496,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
6768412554,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1208689055,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [21]:
df_1.to_csv('aaa.csv')

In [23]:
pd.read_csv('aaa.csv',index_col=0)

,숙박 및 음식점업,도매 및 소매업,건설업,전문 과학 및 기술서비스업,부동산업 및 임대업,농업 임업 및 어업,전기 가스 증기 및 수도사업,제조업,보건업 및 사회복지서비스업,하수폐기물처리 원료재생 및 환경복원,Unnamed: 11,교육서비스업,출판 영상 방송통신 및 정보서비스업,예술 스포츠 및 여가관련 서비스업,금융 및 보험업,운수업,사업시설관리 및 사업지원서비스업,협회 및 단체 수리 및 기타 개인서비스,가구내고용활동,광업
6058191362,0.0,0.031496,0.000000,0.307087,0.000000,0.0,0.000000,0.094488,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.566929,0.000000,0.000000,0.0,0.0
6178603207,0.0,0.001812,0.000000,0.000000,0.016304,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.961957,0.001812,0.018116,0.0,0.0
4098610941,0.0,0.406780,0.144068,0.046610,0.004237,0.0,0.033898,0.152542,0.0,0.0,0.088983,0.0,0.033898,0.0,0.0,0.088983,0.000000,0.000000,0.0,0.0
6038171958,0.0,0.620690,0.000000,0.241379,0.000000,0.0,0.000000,0.068966,0.0,0.0,0.034483,0.0,0.000000,0.0,0.0,0.034483,0.000000,0.000000,0.0,0.0
1288713240,0.0,0.496503,0.000000,0.000000,0.000000,0.0,0.000000,0.489510,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.013986,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8614100470,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
9227588496,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
6768412554,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1208689055,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [20]:
df_1.shape # (3271555, 20)

(365567, 20)

In [ ]:
df_1.reset_index(inplace=True) #pkl 일때는 drop=True를 빼준다

In [ ]:
pickle.dump(df_1, open("aaa.pickle","wb"))